# AFSK Demodulator on PYNQ

This is a series of notebooks on migrating an [AFSK demodulator we developed in Python](https://github.com/mobilinkd/afsk-demodulator/blob/master/afsk-demodulator.ipynb) to FPGA.

![PYNQ Z2 board](03_PYNQ-Z2.png)

The development is being done on a [PYNQ Z2 board](http://www.tul.com.tw/ProductsPYNQ-Z2.html), which uses a Xilinx Zync 7020 SOC.  This chip integrates Xilinx Artix FPGA fabric with a dual-core ARM Cortex A9 processor.  The PYNQ boards are designed to run **Linux** (based on Ubuntu), **Python**, and **Jupyter Notebooks**.  Code developed for the FPGA fabric can be instantiated and accessed directly from Python.  More information on Pynq can be found here: http://www.pynq.io, and the Pynq software stack is documented here: https://pynq.readthedocs.io/

The purpose of this project is to teach myself FPGA development, and to share that process with others.  This is not meant as a "getting started" tutorial, but rather as a step for me, after having done a few basic tutorials, to apply an area that I am quite knowledgable in to one that I am learning about.

The PYNQ Z2 has been a great board for teaching me, a software engineer, how to develop for FPGA.  And the PYNQ environment is an amazing platform for teaching FPGA development to software engineers.  Both are highly recommended.

The PYNQ development environment will allow us to move the demodulator components, step by step, into the FPGA.  This will allow us to explore new areas of the development process and test our code in small, digestible steps.

I will be using this as a *coding kata*, which is to apply repetition to coding practices so that one becomes proficient -- and eventually expert -- in the process and tools used to do development.  I am aiming for proficiency in Vivado and Vivado HLS at this point.


## Prerequisites

For those who would like to follow along with this project to learn FPGA development, I strongly urge you to review the setup guides for your development board, run through the Vivado and Vivado HLS tutorials, and the tutorials on executing code on the FPGA from Python.  I also suggest reading through the AFSK Demodulator notebook linked above to have a high-level understanding of what we are developing.

This is designed for someone with a background in software engineering, having some familiarity with Python and C++.  We will be doing development for the FPGA in C++ (HLS -- high-level synthesis), avoiding Verilog and VHDL (RTL -- register-transfer level) as much as possible.  We will be translating our existing Python code into C++.

You will need:

 - A Zynq development board running Pynq
 - Xilinx Vivado installed
 - Experience with the tutorials on the Pynq web site
 
Before we begin, you are expected to have gained rudimentary knowledge of the AXI interfaces we will be using, passing familiarity with Vivado HLS, generating projects for Zynq in Vivado, and running basic FPGA projects from Python in the Pynq environment.

There are a number of Youtube videos that helped me get started.  These are ones that deserve special mention:

 - [How to set up the PYNQ-Z1](https://www.youtube.com/watch?v=lfw6T9vNM_o)
 - [How to accelerate a Python function with PYNQ](https://www.youtube.com/watch?v=LoLCtSzj9BU)
 - [How to make a custom PYNQ overlay](https://www.youtube.com/watch?v=Dupyek4NUoI&t=791s)
 
The Pynq Z2 is available for around $100 from Newark (March 2019).  It's a great deal for a Zynq development board.  It is especially suited for the work we will be doing here because its audio interface is superior to the more expensive PYNQ Z1.  (We wont actually use the audio interface for this project.)

## Development Steps

We will be migrating the demodulator step by step, in the same steps shown in the Python code from above.  This will give us good practice going through the steps required to generate and use IP for the FPGA.  This repetition help me to learn the mechanics of a process, which is different than understanding the just the concepts.

This project will require that we get into AXI Streams and DMA very quickly, as we will be transferring blocks of audio data to the FPGA for processing, then reading the processed data back.

To start, we will create a basic project that accepts an input stream and returns an output stream.  It will perform a very basic operation to prove that it is actually working.  We will then use that as the basis for further work.  We will be moving each piece, having a comlete working demodulator after each step, but with ever more work being done in the FPGA.

We will:

 1. [Create a basic AXI Streaming solution](intro/afsk-demodulator-fpga.ipynb).
 1. [Migrate the band pass filter to FPGA](band_pass_filter/afsk-demodulator-fpga.ipynb).
 1. [Move the Correlator into the FPGA](correlator/afsk-demodulator-fpga.ipynb).
    1. Move the comparator into the FPGA.
    1. Move the delay line into the FPGA.
    1. Move the XOR logic to the FPGA.
 1. [Move the low-pass filter to the FPGA](project_04/afsk-demodulator-fpga.ipynb).
    1. Move the low-pass filter to the FPGA.
    1. Move the second comparator into the FPGA.
 1. [Move the digital PLL into the FPGA](project_05/afsk-demodulator-fpga.ipynb).
    1. Move the IIR filter into the FPGA.
    1. Move the hysteresis code into the FPGA.
    1. Add a DCD indicator to the project, using a LED from the Pynq Z2 board.
 1. [Move the HDLC code into the FPGA](project_06/afsk-demodulator-fpga.ipynb).
    1. Move the NRZI code to FPGA.
    1. Move the CRC code into the FPGA.

We will leave the AX.25 decoding logic in Python.  But we may add a serial port to our code and add KISS and SLIP encoders to the project so the output can be consumed by Linux AX.25 components.

## Project Organization

Structuring this project is a bit of a challenge.  We are going to have components that are developed and executed on a Linux or Windows workstation (Vivado) and we are going to produce components which are going to be executed on the PYNQ development board.

The structure will be:

 * project_N
   * project_N.ipynb
   * HLS
     - project.hpp
     - project.cpp
     - project_test.cpp
     - solution1
   * Vivado
     - project1
   * Pynq
     - project.ipynb
     - project.bit
     - project.tcl
     - project.hwh
     
We will be using the Vivado .gitignore file from Xilinx: https://www.xilinx.com/Attachment/ar61232.txt with additions for the Pynq components which we will put in source control for demonstration purposes.

Since I am using this project to teach myself FPGA development, I will probably make mistakes early on and explore different ways of doing things.  I may leave these mistakes in with notes on proper implementation later on.